<a href="https://colab.research.google.com/github/JanMStraub/ML-bomberman-project/blob/main/SchwarzMethods1DTalk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# 1-D Poisson mit Additiver & Multiplikativer Schwarz-Präkonditionierung
# Coarse Space (2h), P1-FEM auf [0,1]
import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla
import pandas as pd
from numpy.linalg import cond
from concurrent.futures import ThreadPoolExecutor

def stiffness_matrix(n, h):
    main = 2 * np.ones(n)
    off = -1 * np.ones(n-1)
    return (1/h) * sp.diags([off, main, off], [-1,0,1], format='csr')

def load_vector_constant(n, h):
    return h * np.ones(n)


def define_subdomains(n, width, overlap):
    step = width - overlap
    subs = []
    i = 0
    while True:
        start = i
        end = i + width
        if end >= n:
            # letzte Subdomain: auf die letzten width Knoten setzen
            start = max(0, n - width)
            end = n
            subs.append(np.arange(start, end))
            break
        subs.append(np.arange(start, end))
        i += step
    return subs

def build_restriction_operators(subs, n):
    R_list = []
    # Coarse space: jeden zweiten inneren Knoten
    coarse_idx = np.arange(0, n, 2)
    m = len(coarse_idx)
    R0 = sp.lil_matrix((m, n))
    # injection
    for k, i in enumerate(coarse_idx):
        R0[k, i] = 1.0
    # averaging
    for i in range(1, n, 2):
        left = i // 2
        right = left + 1
        if right < m:
            R0[left, i]  += 0.5
            R0[right, i] += 0.5
    R_list.append(R0.tocsr())

    # lokale R_i
    for sub in subs:
        rows = np.arange(len(sub))
        cols = sub
        data = np.ones(len(sub))
        R_list.append(sp.csr_matrix((data, (rows, cols)), shape=(len(sub), n)))

    return R_list

# Lokale Solver: Faktorisierung von lokalen Matritzen
def build_local_solvers(A, R_list):
    A_loc_list = []
    for R in R_list:
        A_loc_list.append(spla.factorized((R @ A @ R.T).tocsc()))
    return A_loc_list

# PARALLEL Lokale Solver: Faktorisierung von lokalen Matritzen
def build_local_solvers_parallel(A, R_list):
    def local_factorize(R):
        return spla.factorized((R @ A @ R.T).tocsc())

    with ThreadPoolExecutor() as executor:
        A_loc_list = list(executor.map(local_factorize, R_list))

    return A_loc_list


# Additiver Schwarz-Preconditioner ohne parallelisierung
def additive_schwarz_factory(R_list, A_loc_list):
    def A_add_inv(v):
        R0 = R_list[0]
        z = R0.T @ A_loc_list[0](R0 @ v)
        for R, A_i in zip(R_list[1:], A_loc_list[1:]):
            z += R.T @ A_i(R @ v)
        return z
    return A_add_inv
# Additiver Schwarz-Preconditioner mit liste
def additive_schwarz_factory_list(R_list, A_loc_list):
    def A_add_inv(v):
        yi_list = []
        for R, A_i in zip(R_list, A_loc_list):
            yi = R.T @ A_i(R @ v)
            yi_list.append(yi)
        y = sum(yi_list)
        return y
    return A_add_inv

# Additiver Schwarz-Preconditioner mit parallelisierung
def additive_schwarz_factory_parallel(R_list, A_loc_list):
    def A_add_inv(v):
        def local_solve(R, A_i):
            return R.T @ A_i(R @ v)

        with ThreadPoolExecutor() as executor:
            yi_list = list(executor.map(local_solve, R_list, A_loc_list))

        return sum(yi_list)
    return A_add_inv

def conjugate_gradient(A, b, x0=None, tol=1e-8, max_iter=None, M_inv=None):
    n=len(b)
    x = np.zeros(n) if x0 is None else x0.copy()
    r = b - A@x
    z = M_inv(r) if M_inv else r.copy()
    p = z.copy()
    if max_iter is None: max_iter=n
    for k in range(max_iter):
        Ap = A@p
        alpha = (r@z)/(p@Ap)
        x += alpha*p
        r_new = r - alpha*Ap
        if np.linalg.norm(r_new)<tol: return x, k+1
        z_new = M_inv(r_new) if M_inv else r_new.copy()
        beta = (r_new@z_new)/(r@z)
        p = z_new + beta*p
        r, z = r_new, z_new
    return x, max_iter


def exact_solution(n, h):
    x = np.linspace(h, 1 - h, n)
    return 0.5 * x * (1 - x)


# Auswertung
#n_list=[31,63,127,255,511,1023, 2047,4095]
n_list=[31,63,127,255,511]
h_list=[1/(n+1) for n in n_list]
results=[]
for n,h in zip(n_list,h_list):
    A=stiffness_matrix(n,h)
    f=load_vector_constant(n,h)
    subs = define_subdomains(n, width=8, overlap=1)
    R_list=build_restriction_operators(subs,n)
    A_loc_list = build_local_solvers_parallel(A,R_list)

    # Additiv
    A_inv_add=additive_schwarz_factory_parallel(R_list, A_loc_list)
    u_add,it_add=conjugate_gradient(A,f,M_inv=A_inv_add)

    # Referenz CG
    u_CG,it_cg=conjugate_gradient(A,f)

    # error
    u_exact = exact_solution(n, h)
    err_cg = np.linalg.norm(u_CG - u_exact, ord=np.inf)
    err_add = np.linalg.norm(u_add - u_exact, ord=np.inf)

    #tabelle
    results.append({
        "n": n,
        "h": h,
        "Iter. CG": it_cg,
        "Iter. Add": it_add,
        "Fehler Add": f"{err_add:.2e}",
    })

df=pd.DataFrame(results)
print(df.to_markdown(index=False))

|   n |          h |   Iter. CG |   Iter. Add |   Fehler Add |
|----:|-----------:|-----------:|------------:|-------------:|
|  31 | 0.03125    |         16 |          12 |     5.85e-11 |
|  63 | 0.015625   |         32 |          13 |     1.81e-11 |
| 127 | 0.0078125  |         64 |          12 |     1.84e-11 |
| 255 | 0.00390625 |        128 |          12 |     9.26e-12 |
| 511 | 0.00195312 |        256 |          11 |     1.3e-11  |


In [14]:
# Matrix P_ad
def build_additive_operator_matrix(A, R_list):
    n = A.shape[0]
    P_ad = sp.csr_matrix((n, n))

    for R in R_list:
        A_i = R @ A @ R.T
        A_i_inv = spla.inv(A_i.tocsc())
        P_i = R.T @ A_i_inv @ R @ A
        P_ad += P_i

    return P_ad

# Auswertung
n_list=[31,63,127,255,511,1023, 2047,4095]
h_list=[1/(n+1) for n in n_list]
results=[]
for n,h in zip(n_list,h_list):
    A=stiffness_matrix(n,h)
    f=load_vector_constant(n,h)
    subs = define_subdomains(n, width=12, overlap=1)
    R_list=build_restriction_operators(subs,n)
    A_loc_list = build_local_solvers(A,R_list)

    # Condition numbers
    P_ad = build_additive_operator_matrix(A, R_list)
    cond_A = cond(A.todense())
    cond_P_ad = cond(P_ad.todense())

    #tabelle
    results.append({
        "n": n,
        "h": h,
        "cond(A)": cond_A,
        "cond(P_ad)": cond_P_ad
    })

df=pd.DataFrame(results)
print(df.to_markdown(index=False))

|    n |           h |          cond(A) |   cond(P_ad) |
|-----:|------------:|-----------------:|-------------:|
|   31 | 0.03125     |    414.345       |      6.30294 |
|   63 | 0.015625    |   1659.38        |      9.22397 |
|  127 | 0.0078125   |   6639.52        |      9.78969 |
|  255 | 0.00390625  |  26560.1         |     10.8232  |
|  511 | 0.00195312  | 106242           |     10.0629  |
| 1023 | 0.000976562 | 424971           |      9.03969 |
| 2047 | 0.000488281 |      1.69989e+06 |      9.0405  |
| 4095 | 0.000244141 |      6.79955e+06 |     10.3874  |


In [15]:
# Liste der Projektionen P_i (als Funktionen)
def build_Pi_list(A, R_list, A_loc_list):
    Pi_func_list = []
    for R, A_i_inv in zip(R_list, A_loc_list):
        Pi_func_list.append(lambda x, R=R, A_i_inv=A_i_inv: R.T @ A_i_inv(R @ (A @ x)))
    return Pi_func_list


def build_Pi_list_parallel(A, R_list, A_loc_list):
    def build_Pi(R, A_i_inv):
        return lambda x: R.T @ A_i_inv(R @ (A @ x))

    with ThreadPoolExecutor() as executor:
        Pi_func_list = list(executor.map(build_Pi, R_list, A_loc_list))

    return Pi_func_list

def E_mu_factory(Pi_func_list):
    def E_mu(u):
      Eu= (u - Pi_func_list[0](u))
      for P_i in Pi_func_list[1:]:
        Eu = Eu - P_i(Eu)
      return Eu
    return E_mu

def multiplicative_schwarz_factory(A, R_list, A_loc_list):
    def A_mul_inv(v):
        R0 = R_list[0]
        z = R0.T @ A_loc_list[0](R0 @ v)
        for R, A_i_inv in zip(R_list[1:], A_loc_list[1:]):
            z += R.T @ A_i_inv(R @ (v - A @ z))
        return z
    return A_mul_inv

def richardson_fixed_point(E_mu,g_mu,x0=None,tol=1e-10,max_iter=500):
    u=np.zeros_like(g_mu) if x0 is None else x0.copy()
    for k in range(max_iter):
        u_new=E_mu(u)+g_mu
        if np.linalg.norm(u_new-u)<tol: return u_new,k+1
        u=u_new
    return u,max_iter


# Auswertung
#n_list=[31,63,127,255,511,1023, 2047]
#h_list=[1/(n+1) for n in n_list]
results=[]
for n,h in zip(n_list,h_list):
    A=stiffness_matrix(n,h)
    f=load_vector_constant(n,h)
    subs = define_subdomains(n, width=8, overlap=1)
    R_list=build_restriction_operators(subs,n)
    A_loc_list = build_local_solvers_parallel(A,R_list)
    Pi_list = build_Pi_list_parallel(A, R_list,A_loc_list)
    # Additiv
    A_inv_add=additive_schwarz_factory(R_list, A_loc_list)
    u_add,it_add=conjugate_gradient(A,f,M_inv=A_inv_add)

    # Multiplikativ
    A_inv_mul=multiplicative_schwarz_factory(A, R_list, A_loc_list)
    g_mu=A_inv_mul(f)
    E_mu=E_mu_factory(Pi_list)
    u_mul,it_mul=richardson_fixed_point(E_mu,g_mu)

    # Referenz CG
    u_CG,it_cg=conjugate_gradient(A,f)

    # error
    u_exact = exact_solution(n, h)
    err_add = np.linalg.norm(u_add - u_exact, ord=np.inf)
    err_mul = np.linalg.norm(u_mul - u_exact, ord=np.inf)

    #tabelle
    results.append({
        "n": n,
        "h": h,
        "Iter. CG": it_cg,
        "Iter. Add": it_add,
        "Iter. Mul": it_mul,
        "Fehler Add": f"{err_add:.2e}",
        "Fehler Mul": f"{err_mul:.2e}"
    })

df=pd.DataFrame(results)
print(df.to_markdown(index=False))




|    n |           h |   Iter. CG |   Iter. Add |   Iter. Mul |   Fehler Add |   Fehler Mul |
|-----:|------------:|-----------:|------------:|------------:|-------------:|-------------:|
|   31 | 0.03125     |         16 |          12 |           3 |     5.85e-11 |     2.78e-17 |
|   63 | 0.015625    |         32 |          13 |           3 |     1.81e-11 |     5.69e-16 |
|  127 | 0.0078125   |         64 |          12 |           3 |     1.84e-11 |     5e-16    |
|  255 | 0.00390625  |        128 |          12 |           3 |     9.26e-12 |     1.39e-16 |
|  511 | 0.00195312  |        256 |          11 |           3 |     1.3e-11  |     1.57e-15 |
| 1023 | 0.000976562 |        512 |          11 |           3 |     1.41e-12 |     4.16e-16 |
| 2047 | 0.000488281 |       1024 |          11 |           3 |     1.11e-12 |     7.66e-15 |
| 4095 | 0.000244141 |       2048 |          10 |           3 |     3.61e-12 |     2.3e-15  |


In [16]:
def gmres_saad(A, b, x0=None, tol=1e-10, max_iter=None, M_inv=None):
    """
    GMRES based on Saad's Algorithm 6.5.
    Solves Ax = b or preconditioned system M^{-1}Ax = M^{-1}b
    """
    n = len(b)
    x = np.zeros(n) if x0 is None else x0.copy()

    r0 = b - A @ x
    r0 = M_inv(r0) if M_inv else r0
    beta = np.linalg.norm(r0)
    if beta < tol:
        return x, 0

    if max_iter is None:
        max_iter = n

    V = np.zeros((n, max_iter + 1))
    H = np.zeros((max_iter + 1, max_iter))
    e1 = np.zeros(max_iter + 1)
    e1[0] = 1.0

    V[:, 0] = r0 / beta
    g = beta * e1

    for j in range(max_iter):
        w = A @ V[:, j]
        w = M_inv(w) if M_inv else w

        for i in range(j + 1):
            H[i, j] = np.dot(V[:, i], w)
            w -= H[i, j] * V[:, i]

        H[j + 1, j] = np.linalg.norm(w)
        if H[j + 1, j] != 0:
            V[:, j + 1] = w / H[j + 1, j]
        else:
            # lucky breakdown
            y = np.linalg.lstsq(H[:j + 1, :j + 1], g[:j + 1], rcond=None)[0]
            return x + V[:, :j + 1] @ y, j + 1

        # solve least squares problem
        y = np.linalg.lstsq(H[:j + 2, :j + 1], g[:j + 2], rcond=None)[0]
        x_new = x + V[:, :j + 1] @ y
        r_new = b - A @ x_new
        res_norm = np.linalg.norm(M_inv(r_new) if M_inv else r_new)

        if res_norm < tol:
            return x_new, j + 1

    return x_new, max_iter

    return x, iter_total


In [17]:
# Auswertung
#n_list=[31,63,127,255,511,1023]
#h_list=[1/(n+1) for n in n_list]
results=[]
for n,h in zip(n_list,h_list):
    A=stiffness_matrix(n,h)
    f=load_vector_constant(n,h)

    #subs = define_subdomains_fixed_intervals(n, width=16, overlap=2)
    subs = define_subdomains(n, width=9, overlap=1)
    R_list=build_restriction_operators(subs,n)
    A_loc_list = build_local_solvers_parallel(A,R_list)
    Pi_list = build_Pi_list_parallel(A, R_list,A_loc_list)
    # Additiv
    A_inv_add=additive_schwarz_factory(R_list, A_loc_list)
    u_add,it_add=gmres_saad(A,f,max_iter=500,M_inv=A_inv_add)

    # Multiplikativ
    A_inv_mul=multiplicative_schwarz_factory(A, R_list, A_loc_list)
    u_mul,it_mul=gmres_saad(A,f,max_iter=500,M_inv=A_inv_mul)

    # ohne Preconditioner
    u_gmres,it_gmres=gmres_saad(A,f,max_iter=600)

    # error
    u_exact = exact_solution(n, h)
    err_gmres = np.linalg.norm(u_gmres - u_exact, ord=np.inf)
    err_add = np.linalg.norm(u_add - u_exact, ord=np.inf)
    err_mul = np.linalg.norm(u_mul - u_exact, ord=np.inf)

    #tabelle
    results.append({
        "n": n,
        "h": h,
        "Iter. GMRES": it_gmres,
        "Iter. Add": it_add,
        "Iter. Mul": it_mul,
        "Fehler GMRES": f"{err_gmres:.2e}",
        "Fehler Add": f"{err_add:.2e}",
        "Fehler Mul": f"{err_mul:.2e}"
    })

df=pd.DataFrame(results)
print(df.to_markdown(index=False))


|    n |           h |   Iter. GMRES |   Iter. Add |   Iter. Mul |   Fehler GMRES |   Fehler Add |   Fehler Mul |
|-----:|------------:|--------------:|------------:|------------:|---------------:|-------------:|-------------:|
|   31 | 0.03125     |            16 |          13 |           2 |       2.78e-16 |     1.73e-12 |     1.67e-16 |
|   63 | 0.015625    |            32 |          13 |           2 |       1.45e-15 |     6.14e-12 |     5.55e-17 |
|  127 | 0.0078125   |            64 |          13 |           2 |       4.24e-14 |     2.84e-12 |     8.33e-17 |
|  255 | 0.00390625  |           128 |          12 |           2 |       5.06e-14 |     4.08e-12 |     8.47e-16 |
|  511 | 0.00195312  |           256 |          11 |           2 |       7.92e-13 |     5.49e-12 |     9.71e-16 |
| 1023 | 0.000976562 |           512 |          10 |           2 |       4.17e-12 |     9.66e-12 |     1.93e-15 |
| 2047 | 0.000488281 |           600 |           9 |           2 |       0.082    |     